In [1]:
import sys
sys.path.append("..")

from opdynamics.simulation.simulation import initialize_model
from opdynamics.utils.model_modification import modify_model

In [2]:
params = {
    "graph_type": "barabasi",
    "network_size": 500,
    "memory_size": 160,
    "code_length": 5,
    "kappa": 30,
    "lambd": 0,
    "alpha": 0,
    "omega": 0, 
    "gamma": 1,
    "preferential_attachment": 2,
    "polarization_grouping_type": 1,
    "T": 500,
    "num_repetitions": 50,
    "results_path": "results"
}

model = initialize_model(**params)
# model = modify_model(model)

{}


In [15]:
from opdynamics.utils.types import Parameters
import hashlib

def param_to_hash(params: Parameters) -> str:
    param_tuple = tuple(params.values())
    string = str(param_tuple).encode("utf-8")
    print(string)
    return str(hashlib.sha256(string).hexdigest())

In [16]:
param_to_hash(params)

b"('barabasi', 500, 160, 5, 30, 0, 0, 0, 1, 2, 1, 500, 50, 'results')"


'a0cb6806b530ba4cee4bdd93835952e2abcd09e0ea0285e2cbf2c92d2fccc79a'

In [31]:
f = open("results/last_run.txt", "w")
f.write("0")
f.close()

AttributeError: 'int' object has no attribute 'close'

In [30]:
f = open("results/last_run.txt", "r")
int(f.read())

0